In [3]:
import numpy as np 
import pandas as pd 

class Die(): 
    
    '''This class takes a die of N number of sides. Each side is initially evenly weighted but 
        you can change the weight of the sides. This class also has functions that will roll the die 
        and show the results as well.'''
    
    def __init__(self, face): 
        "Intializes a numpy as the faces of the die."
    #takes numpy as an argument and has an error if not 
        if not isinstance(face, np.ndarray):
            raise TypeError("Input must be a NumPy array.")
            
    #raises a ValueError if the faces are not distinct     
        if len(np.unique(face)) != len(face): 
            raise ValueError("All face values must be distinct.")
    
        weights = np.ones(len(face), dtype = float)
        
        # Save to a private DataFrame with faces as index
        self.__data = pd.DataFrame({'weight': weights}, index=face)
        
    def change_weight(self, face, new_weight):
        "A method to change the weight of a single side."
     # Check if face exists in the index
        if face not in self.__data.index:
            raise IndexError(f"Face '{face}' not found in die faces.")   
            
        try: 
            numeric_weight = float(new_weight) 
        except (TypeError, ValueError):
            raise TypeError("New weight must be a number or castable to a number.")
        
        # Assign the new weight
        self.__data.loc[face, 'weight'] = numeric_weight
        
    def roll_die(self, num_rolls=1):
        "A method to roll the die one or more times."
        # Validate num_rolls is a positive integer
        if not isinstance(num_rolls, int) or num_rolls < 1:
            raise ValueError("Number of rolls must be a positive integer.")
        
        # get the faces and weights 
        faces = self.__data.index.to_numpy()
        weights = self.__data['weight'].to_numpy()
        
        # Normalize weights to convert them into probabilities
        probabilities = weights / weights.sum()

        # Use numpy to randomly choose faces based on weights
        rolls = np.random.choice(faces, size=num_rolls, p=probabilities)
        
        return rolls.tolist()

    def show_die(self):
        "A method to show the die’s current state."
      #Returns a copy of the private die data frame.
      return self.__data.copy()
        
        

In [32]:
faces = np.array([1, 2, 3, 4, 5, 6])
my_die = Die(faces)


In [33]:
print("Initial die state:")
print(my_die.show_die())

Initial die state:
   weight
1     1.0
2     1.0
3     1.0
4     1.0
5     1.0
6     1.0


In [34]:
my_die.change_weight(6, 10)
print("\nAfter changing weight of face 6:")
print(my_die.show_die())


After changing weight of face 6:
   weight
1     1.0
2     1.0
3     1.0
4     1.0
5     1.0
6    10.0


In [35]:
print("\nRolling die 5 times:")
print(my_die.roll_die(5))


Rolling die 5 times:
[6, 6, 6, 2, 6]


In [38]:
import numpy as np 
import pandas as pd 

class Game():
    
    """The class takes multiple die in a list. From there, you can roll the die all together to play.
    You can also print out the game results in a data frame."""
    
    def __init__(self, dice_list):
        
        self.dice_list = dice_list
        self.__results = None 
        
    def play(self, num_rolls):
        
        if not isinstance(num_rolls, int) or num_rolls<1:
            raise ValueError("Number of rolls must be a positive integer.")
            
        results = {}
            
        for i, die in enumerate(self.dice_list):
                results[i] = die.roll_die(num_rolls)
                
        self.__results = pd.DataFrame(results) 
        self.__results.index.name = 'roll'
        self.__results.index += 1
        
    def game_results(self, form='wide'): 
        "A method to show the user the results of the most recent play."
        if self.__results is None:
            return None 
        
        if form == 'wide':
            return self.__results.copy()
        elif form == 'narrow':
            stacked = self.__results.stack()
            df = stacked.to_frame(name = 'result') 
            df = df.rename_axis(['roll', 'die'])
            df = df.reset_index()
            df = df.set_index(['roll', 'die'])
            return df 
                              
        else:
            raise ValueError("Invalid. Choose narrow or wide.")
        

In [23]:
die_face = np.array(['a','b','c','d'])
die1 = Die(die_face)
die2 = Die(die_face)
die3 = Die(die_face) 

In [24]:
game = Game([die1, die2, die3])

In [25]:
game.play(5)

In [44]:
game.game_results('narrow')

result
roll die       
1    0        d
     1        d
     2        c
2    0        a
     1        a
     2        d
3    0        c
     1        a
     2        c
4    0        b
     1        a
     2        d
5    0        d
     1        c
     2        b

In [37]:
game.game_results('wide')

,0,1,2
roll,,,
1,d,d,c
2,a,a,d
3,c,a,c
4,b,a,d
5,d,c,b


In [46]:
import pandas as pd
from collections import Counter

class Analyzer():
    '''An Analyzer object takes the results of a single game and computes various descriptive 
    statistical properties about it.'''


    def __init__(self, game):
        "Takes a game object as its input parameter and gives an error if not a game object"
        if not isinstance(game, Game):
            raise ValueError("Error. Expected a Game object.")
        
        self.game = game 
        self.results = game.game_results()
        
            
    def jackpot(self):
        "Checks to see how many times all the faces have the same value."
        
        jackpots = self.results.nunique(axis=1) == 1 
        return jackpots.sum()
    
    def roll_count(self):
        "Counts how many times a given face is rolled in each event."
        
        face_counts = self.results.apply(lambda row: row_value_counts, aixs=1).fillna(0).astype(int)
        face_counts.index_name = 'row' 
        face_counts.columns.name = 'face' 
        return face_counts
        
    def combo_count(self):
        
        combos = self.results.apply(lambda row: tuple(sorted(row)), axis=1)
        combo_counts = combos.value_counts().to_frame(name='count')
        combo_counts.index.name = 'combination'
        return combo_counts
    
    def permutation_count(self):
        
        permutations = self.results.apply(lambda row: tuple(row), axis=1)
        perm_counts = permutations.value_counts().to_frame(name='count')
        perm_counts.index.name = 'permutation'
        return perm_counts